# **1. SCRAPPING DATA**

#Import Library
Langkah pertama yang perlu dilakukan adalah mengimpor library yang akan digunakan. Kita akan menggunakan:
*   **pandas** untuk memproses dan menyimpan data dalam bentuk tabel.
*   **requests** untuk mengirim permintaan HTTP,
*   **BeautifulSoup** dari library bs4 untuk melakukan parsing terhadap konten HTML,
pandas untuk memproses dan menyimpan data dalam bentuk tabel.



In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Menentukan URL dan Menyusun Header Permintaan
Langkah berikutnya adalah menetapkan URL halaman web yang akan diambil datanya (scraping). Dalam contoh ini, targetnya adalah halaman pertama dari hasil pencarian pekerjaan Data Scientist di situs pekerjaan Jobstreet. Disini saya hanya ingin mengambil data lowongan pekerjaan dengan gaji minimal 2 juta, full time, dan terdaftar di 30 hari terakhir.

In [18]:
url = 'https://id.jobstreet.com/id/Data-Scientist-jobs/full-time?daterange=31&salaryrange=2000000-&salarytype=monthly&workarrangement=1%2C3%2C2'

Setelah URL ditentukan, kita juga perlu menyiapkan header untuk permintaan HTTP. Header ini berguna agar permintaan yang dikirim tampak seperti berasal dari browser asli, sehingga dapat menghindari pemblokiran karena terdeteksi sebagai aktivitas bot.

In [19]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
}

- **User-Agent** adalah informasi yang menunjukkan jenis browser yang digunakan. Dengan menyertakan User-Agent dalam header, kita memberi tahu server bahwa permintaan berasal dari browser tertentu—misalnya Google Chrome—sehingga lebih kecil kemungkinannya untuk ditolak.

#Mengirim Permintaan HTTP
Setelah URL dan header disiapkan, kita dapat mengirimkan permintaan ke website menggunakan requests.get() untuk mengambil konten dari halaman tersebut.

In [20]:
s = requests.session()
s.headers.update(headers)

page = s.get(url)

- **Session** memungkinkan kita mengelola permintaan dan header secara lebih efisien, terutama jika kita melakukan beberapa permintaan ke situs yang sama.
- **get()** adalah metode untuk mengirim permintaan HTTP GET ke URL yang telah ditentukan dan menerima respons berupa isi halaman.

# Memeriksa Status Permintaan (Request Status)
Sebelum memproses data dari halaman web, sangat penting untuk memastikan bahwa permintaan HTTP yang kita kirim berhasil. Ini bisa diketahui melalui kode status (status code) yang diberikan dalam respons.

In [21]:
if page.status_code == 200:
    print("Request berhasil!")
else:
    print(f"Request gagal dengan kode status: {page.status_code}")

Request berhasil!



status_code: Merupakan atribut dari respons HTTP yang menunjukkan status dari permintaan.

- 200 menandakan bahwa permintaan berhasil dan halaman berhasil dimuat.

- Kode selain 200, seperti 404 (Not Found) atau 403 (Forbidden), menunjukkan bahwa terjadi kesalahan—entah karena URL tidak valid, halaman tidak tersedia, atau akses dibatasi.

Sebagai langkah verifikasi, kita bisa menggunakan kondisi page.status_code == 200. Jika hasilnya bukan 200, maka perlu dilakukan pengecekan lebih lanjut terhadap URL atau penyesuaian pada konfigurasi permintaan.

#Memproses HTML dengan BeautifulSoup
Jika permintaan HTTP berhasil (status code 200), langkah selanjutnya adalah mengambil isi halaman dan memprosesnya menggunakan BeautifulSoup.

In [22]:
soup = BeautifulSoup(page.text, "lxml")
joblist = soup.find('div', class_='_32fem00 _1nh354wo')

- **BeautifulSoup** berfungsi untuk mengurai (parse) konten HTML dan mengubahnya menjadi objek Python yang mudah ditelusuri dan dimanipulasi.

- Metode **find()** digunakan untuk mencari elemen HTML tertentu berdasarkan tag dan class. Dalam konteks ini, kita akan mencari elemen (div) yang memuat daftar lowongan pekerjaan.

Dengan pendekatan ini, kita dapat mengekstrak informasi penting dari halaman web secara terstruktur dan efisien.

In [23]:
# print(joblist.prettify())

#Mengambil Data dari Setiap Pekerjaan
Setelah kita menemukan daftar pekerjaan, langkah selanjutnya adalah mengambil detail dari setiap pekerjaan yang ada pada halaman tersebut.

In [24]:
jobs_data = []
for artikel in joblist.find_all('article', {'class': '_32fem00 _32fem01 _1nh354w98 _1nh354w8t _1nh354w84 _1nh354w7p _1nh354wbg _1nh354wb1 _1nh354wac _1nh354w9x _1nh354wi _1nh354w6c _1nh354w5g _1qx54cib _1qx54ci9 _1qx54cia gyz43x10 gyz43x13 _1nh354w34 _1nh354w37'}):
    posisi = artikel.find('div', {'class': '_32fem00 _1nh354w5i _1nh354w54'})
    perusahaan = artikel.find(attrs={'data-automation':'jobCompany'})
    lokasi = artikel.find(attrs={'data-automation':'jobLocation'})
    gaji = artikel.find('span', {'class':'_32fem00 wo05ot2 _1nh354w50 _1nh354w0 _1nh354ws wo05ot4'})
    waktu = artikel.find(attrs={'data-automation':'jobListingDate'})
    deskripsi = artikel.find('span', {'class':'_32fem00 _1nh354w50 gyz43x0 gyz43x1 gyz43x1u gyz43x6 _1lwlriv4'})

**Menelusuri dan Mengekstrak Informasi Detail Pekerjaan**

- find_all(): Digunakan untuk menemukan semua elemen <article> yang sesuai dengan class tertentu. Ini memungkinkan kita mengakses seluruh daftar lowongan pekerjaan yang ditampilkan pada halaman.

Setelah setiap elemen artikel ditemukan, kita dapat mengekstrak berbagai informasi penting dari masing-masing lowongan, seperti:

*   Posisi: Nama atau judul pekerjaan yang ditawarkan.
*   Perusahaan: Nama perusahaan yang membuka lowongan.
*   Lokasi: Lokasi penempatan kerja.
*   Gaji: Informasi mengenai kisaran atau detail gaji (jika tersedia).
*   Waktu: Kapan lowongan tersebut diunggah atau diperbarui.
*   Deskripsi: Ringkasan singkat mengenai tanggung jawab dan kualifikasi pekerjaan.

Semua informasi ini bisa diambil dengan memanfaatkan metode seperti .find() atau .text pada masing-masing elemen HTML dalam struktur hasil parsing.

##Menangani Elemen yang Tidak Ditemukan
Saat melakukan web scraping, tidak semua elemen HTML yang kita cari selalu tersedia. Oleh karena itu, penting untuk menangani kemungkinan elemen tidak ditemukan agar program tetap berjalan tanpa error.

In [25]:
posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
perusahaan_text = perusahaan.get_text() if perusahaan else "Perusahaan tidak ditemukan"
lokasi_text = lokasi.get_text() if lokasi else "Lokasi tidak ditemukan"
gaji_text = gaji.get_text() if gaji else "Gaji tidak disebutkan"
waktu_text = waktu.get_text() if waktu else "Waktu tidak ditemukan"
deskripsi_text = deskripsi.get_text() if deskripsi else "Deskripsi tidak ditemukan"

Salah satu pendekatan umum adalah menggunakan operator ternary untuk memberikan nilai default jika elemen tidak ditemukan.

- get_text() digunakan untuk mengambil isi teks dari suatu elemen HTML.

- Jika elemen tidak tersedia (misalnya None), kita bisa memberikan nilai pengganti, seperti: "Posisi tidak ditemukan"

Dengan teknik ini, proses ekstraksi data tetap berjalan mulus meskipun ada data yang hilang atau tidak lengkap pada halaman yang di-scrape.

## Menyimpan Data ke List
Setelah semua data berhasil diambil, kita menyimpan data tersebut dalam sebuah list jobs_data.

In [26]:
jobs_data.append({
    "Posisi": posisi_text,
    "Perusahaan": perusahaan_text,
    "Lokasi": lokasi_text,
    "Gaji": gaji_text,
    "Waktu": waktu_text,
    "Deskripsi": deskripsi_text
})

- append() digunakan untuk menambahkan setiap item data pekerjaan ke dalam list bernama jobs_data.

Setiap data yang ditambahkan biasanya disimpan dalam bentuk dictionary, yang berisi pasangan key-value seperti posisi, perusahaan, lokasi, gaji, waktu, dan deskripsi.

## *Code* lengkap pengambilan data (1 halaman web)

In [27]:
jobs_data = []
for artikel in joblist.find_all('article', {'class': '_32fem00 _32fem01 _1nh354w98 _1nh354w8t _1nh354w84 _1nh354w7p _1nh354wbg _1nh354wb1 _1nh354wac _1nh354w9x _1nh354wi _1nh354w6c _1nh354w5g _1qx54cib _1qx54ci9 _1qx54cia gyz43x10 gyz43x13 _1nh354w34 _1nh354w37'}):
    posisi = artikel.find('div', {'class': '_32fem00 _1nh354w5i _1nh354w54'})
    perusahaan = artikel.find(attrs={'data-automation':'jobCompany'})
    lokasi = artikel.find(attrs={'data-automation':'jobLocation'})
    gaji = artikel.find('span', {'class':'_32fem00 wo05ot2 _1nh354w50 _1nh354w0 _1nh354ws wo05ot4'})
    waktu = artikel.find(attrs={'data-automation':'jobListingDate'})
    deskripsi = artikel.find('span', {'class':'_32fem00 _1nh354w50 gyz43x0 gyz43x1 gyz43x1u gyz43x6 _1lwlriv4'})

    posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
    perusahaan_text = perusahaan.get_text() if perusahaan else "Perusahaan tidak ditemukan"
    lokasi_text = lokasi.get_text() if lokasi else "Lokasi tidak ditemukan"
    gaji_text = gaji.get_text() if gaji else "Gaji tidak disebutkan"
    waktu_text = waktu.get_text() if waktu else "Waktu tidak ditemukan"
    deskripsi_text = deskripsi.get_text() if deskripsi else "Deskripsi tidak ditemukan"

    print(posisi_text)
    print(perusahaan_text)
    print(lokasi_text)
    print(gaji_text)
    print(waktu_text)
    print(deskripsi_text)
    print("==========================================")

    jobs_data.append({
        "Posisi": posisi_text,
        "Perusahaan": perusahaan_text,
        "Lokasi": lokasi_text,
        "Gaji": gaji_text,
        "Waktu": waktu_text,
        "Deskripsi": deskripsi_text
    })

Data Scientist
PT Ninety Nine Dotco
Jakarta Raya
Gaji tidak disebutkan
5 hari yang lalu
We are looking for a data scientist who is equally comfortable writing SQL queries and ML models as they are stepping into a room with stakeholders...
Junior Data Scientist
PT Fata Organa Solusi
Tangerang
Rp 8.500.000 – Rp 10.500.000 per month
1 hari yang lalu
Career growth opportunity in a rapidly growing company
Senior Data Scientist
PT Kamoro Maxima Integra
Jakarta Raya
Gaji tidak disebutkan
9 hari yang lalu
Minimum 5 years experience as Data Scientist, strong in predictive modelling, python, cloud-based ML (Azure). Willing to travel.
Data Scientist (Analytics)
PT Solusi Transportasi Indonesia
Jakarta Raya
Gaji tidak disebutkan
16 hari yang lalu
You will use data to understand user needs and be the de-facto Voice of the Customer (users, driver-partners, merchants, agents) across GrabFin's...
Data Analyst
Pengiklan Anonim
Tangerang
Gaji tidak disebutkan
5 hari yang lalu
Budaya kerja kekeluargaan
I

#Menyimpan Data dari Satu Halaman ke CSV
Setelah data dari satu halaman terkumpul, kita bisa menyimpan hasilnya ke dalam bentuk file CSV.

In [28]:
df_jobs = pd.DataFrame(jobs_data)
#df_jobs.to_csv('Data_Job_Listings.csv', index=False)

- **pd.DataFrame()** digunakan untuk mengonversi list jobs_data (yang berisi kumpulan dictionary data pekerjaan) menjadi objek DataFrame dari Pandas.
DataFrame ini memudahkan kita dalam melakukan analisis, pemfilteran, visualisasi, maupun ekspor data ke format lain.
- **to_csv()** berfungsi untuk menyimpan DataFrame ke dalam file CSV, sehingga data dapat digunakan kembali di luar Python, seperti di Excel atau aplikasi pengolah data lainnya.

In [29]:
df_jobs.head()

,Posisi,Perusahaan,Lokasi,Gaji,Waktu,Deskripsi
0,Data Scientist,PT Ninety Nine Dotco,Jakarta Raya,Gaji tidak disebutkan,5 hari yang lalu,We are looking for a data scientist who is equ...
1,Junior Data Scientist,PT Fata Organa Solusi,Tangerang,Rp 8.500.000 – Rp 10.500.000 per month,1 hari yang lalu,Career growth opportunity in a rapidly growing...
2,Senior Data Scientist,PT Kamoro Maxima Integra,Jakarta Raya,Gaji tidak disebutkan,9 hari yang lalu,"Minimum 5 years experience as Data Scientist, ..."
3,Data Scientist (Analytics),PT Solusi Transportasi Indonesia,Jakarta Raya,Gaji tidak disebutkan,16 hari yang lalu,You will use data to understand user needs and...
4,Data Analyst,Pengiklan Anonim,Tangerang,Gaji tidak disebutkan,5 hari yang lalu,Budaya kerja kekeluargaan


#Scrapping Multi Halaman (Halaman 1-4)
Setelah berhasil scraping satu halaman, kita bisa memperluas scraping ke beberapa halaman sekaligus. Kita bisa menggunakan loop untuk mengambil data dari halaman 1 hingga halaman tertentu. Pada kasus ini karena pada website Jobstreet hanya tersedia hingga halaman ke 4, maka range halaman yang akan di scrapping adalah 5 sebagai batas atasnya.

In [30]:
jobs_data = []

for i in range(1,5):
  url = f'https://id.jobstreet.com/id/Data-Scientist-jobs/full-time?daterange=31&page={i}&salaryrange=2000000-&salarytype=monthly&workarrangement=1%2C3%2C2'

  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
    }

  s = requests.session()
  s.headers.update(headers)

  try:
      page = s.get(url)
      soup = BeautifulSoup(page.text, "lxml")
      joblist = soup.find('div', class_='_32fem00 _1nh354wo')

      for artikel in joblist.find_all('article', {'class': '_32fem00 _32fem01 _1nh354w98 _1nh354w8t _1nh354w84 _1nh354w7p _1nh354wbg _1nh354wb1 _1nh354wac _1nh354w9x _1nh354wi _1nh354w6c _1nh354w5g _1qx54cib _1qx54ci9 _1qx54cia gyz43x10 gyz43x13 _1nh354w34 _1nh354w37'}):
          posisi = artikel.find('div', {'class': '_32fem00 _1nh354w5i _1nh354w54'})
          perusahaan = artikel.find(attrs={'data-automation':'jobCompany'})
          lokasi = artikel.find(attrs={'data-automation':'jobLocation'})
          gaji = artikel.find('span', {'class':'_32fem00 wo05ot2 _1nh354w50 _1nh354w0 _1nh354ws wo05ot4'})
          waktu = artikel.find(attrs={'data-automation':'jobListingDate'})
          deskripsi = artikel.find('span', {'class':'_32fem00 _1nh354w50 gyz43x0 gyz43x1 gyz43x1u gyz43x6 _1lwlriv4'})

          posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
          perusahaan_text = perusahaan.get_text() if perusahaan else "Perusahaan tidak disebutkan"
          lokasi_text = lokasi.get_text() if lokasi else "Lokasi tidak ditemukan"
          gaji_text = gaji.get_text() if gaji else "Gaji tidak disebutkan"
          waktu_text = waktu.get_text() if waktu else "Waktu tidak ditemukan"
          deskripsi_text = deskripsi.get_text() if deskripsi else "Deskripsi tidak ditemukan"

          jobs_data.append({
            "Posisi": posisi_text,
            "Perusahaan": perusahaan_text,
            "Lokasi": lokasi_text,
            "Gaji": gaji_text,
            "Waktu": waktu_text,
            "Deskripsi": deskripsi_text
          })

          print(posisi_text)
          print(perusahaan_text)
          print(lokasi_text)
          print(gaji_text)
          print(waktu_text)
          print(deskripsi_text)
          print("===============================")
  except requests.exceptions.RequestException as e:
      print(f"Error: {e}")

Data Scientist
PT Ninety Nine Dotco
Jakarta Raya
Gaji tidak disebutkan
5 hari yang lalu
We are looking for a data scientist who is equally comfortable writing SQL queries and ML models as they are stepping into a room with stakeholders...
Junior Data Scientist
PT Fata Organa Solusi
Tangerang
Rp 8.500.000 – Rp 10.500.000 per month
1 hari yang lalu
Career growth opportunity in a rapidly growing company
Senior Data Scientist
PT Kamoro Maxima Integra
Jakarta Raya
Gaji tidak disebutkan
9 hari yang lalu
Minimum 5 years experience as Data Scientist, strong in predictive modelling, python, cloud-based ML (Azure). Willing to travel.
Data Scientist (Analytics)
PT Solusi Transportasi Indonesia
Jakarta Raya
Gaji tidak disebutkan
16 hari yang lalu
You will use data to understand user needs and be the de-facto Voice of the Customer (users, driver-partners, merchants, agents) across GrabFin's...
Data Analyst
Pengiklan Anonim
Tangerang
Gaji tidak disebutkan
5 hari yang lalu
Budaya kerja kekeluargaan
I

In [31]:
# Tampilkan hasil
df_jobs_full = pd.DataFrame(jobs_data)
df_jobs_full.head()

,Posisi,Perusahaan,Lokasi,Gaji,Waktu,Deskripsi
0,Data Scientist,PT Ninety Nine Dotco,Jakarta Raya,Gaji tidak disebutkan,5 hari yang lalu,We are looking for a data scientist who is equ...
1,Junior Data Scientist,PT Fata Organa Solusi,Tangerang,Rp 8.500.000 – Rp 10.500.000 per month,1 hari yang lalu,Career growth opportunity in a rapidly growing...
2,Senior Data Scientist,PT Kamoro Maxima Integra,Jakarta Raya,Gaji tidak disebutkan,9 hari yang lalu,"Minimum 5 years experience as Data Scientist, ..."
3,Data Scientist (Analytics),PT Solusi Transportasi Indonesia,Jakarta Raya,Gaji tidak disebutkan,16 hari yang lalu,You will use data to understand user needs and...
4,Data Analyst,Pengiklan Anonim,Tangerang,Gaji tidak disebutkan,5 hari yang lalu,Budaya kerja kekeluargaan


In [32]:
# Menyimpan file full
df_jobs_full.to_csv('Full_Data_Job_Listings.csv', index=False)

In [33]:
# Melihat dimensi data
df_jobs_full.shape

(108, 6)

# **2. DATA PREPOCESSING**

## Normalisasi (Lowercase & Hapus Karakter Khusus)


Normalisasi digunakan agar format teks seragam (huruf kecil semua dan tanpa simbol).

In [34]:
import re


In [35]:
# Mendefinisikan kembali dataset yang telah diperoleh sebelumnya agar lebih mudah dieksekusi
df = df_jobs_full.copy()
df

,Posisi,Perusahaan,Lokasi,Gaji,Waktu,Deskripsi
0,Data Scientist,PT Ninety Nine Dotco,Jakarta Raya,Gaji tidak disebutkan,5 hari yang lalu,We are looking for a data scientist who is equ...
1,Junior Data Scientist,PT Fata Organa Solusi,Tangerang,Rp 8.500.000 – Rp 10.500.000 per month,1 hari yang lalu,Career growth opportunity in a rapidly growing...
2,Senior Data Scientist,PT Kamoro Maxima Integra,Jakarta Raya,Gaji tidak disebutkan,9 hari yang lalu,"Minimum 5 years experience as Data Scientist, ..."
3,Data Scientist (Analytics),PT Solusi Transportasi Indonesia,Jakarta Raya,Gaji tidak disebutkan,16 hari yang lalu,You will use data to understand user needs and...
4,Data Analyst,Pengiklan Anonim,Tangerang,Gaji tidak disebutkan,5 hari yang lalu,Budaya kerja kekeluargaan
...,...,...,...,...,...,...
103,Data & Application Operation Center,PT Jaya Teknik Indonesia,Jakarta Raya,Gaji tidak disebutkan,25 hari yang lalu,Pengalaman pekerjaan bidang IT selama minimal ...
104,Product Head,PT Cybertrend Intrabuana,Pasar Minggu,Gaji tidak disebutkan,22 hari yang lalu,"CYBERTREND, as one of key player Data Science ..."
105,Product Innovation Excellence Head,PT Cybertrend Intrabuana,Pasar Minggu,Gaji tidak disebutkan,22 hari yang lalu,"At CYBERTREND, we believe that every company c..."
106,Senior Technology Architect,Finns Beach Club,Badung,Gaji tidak disebutkan,6 hari yang lalu,FINNS World's Best Beach Club is more than a b...


In [36]:
def normalize_text(text):
    text = text.lower()  # Mengubah text menjadi huruf kecil
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Menghapus karakter non-alfanumerik
    return text

# Menerapkan ke kolom "Deskripsi"
df['Deskripsi_Norm'] = df['Deskripsi'].apply(normalize_text)

# Tampilkan hasil
print("Hasil Setelah Normalisasi:")
display(df[['Deskripsi', 'Deskripsi_Norm']])

Hasil Setelah Normalisasi:


,Deskripsi,Deskripsi_Norm
0,We are looking for a data scientist who is equ...,we are looking for a data scientist who is equ...
1,Career growth opportunity in a rapidly growing...,career growth opportunity in a rapidly growing...
2,"Minimum 5 years experience as Data Scientist, ...",minimum 5 years experience as data scientist s...
3,You will use data to understand user needs and...,you will use data to understand user needs and...
4,Budaya kerja kekeluargaan,budaya kerja kekeluargaan
...,...,...
103,Pengalaman pekerjaan bidang IT selama minimal ...,pengalaman pekerjaan bidang it selama minimal ...
104,"CYBERTREND, as one of key player Data Science ...",cybertrend as one of key player data science a...
105,"At CYBERTREND, we believe that every company c...",at cybertrend we believe that every company ca...
106,FINNS World's Best Beach Club is more than a b...,finns worlds best beach club is more than a be...


## Penghapusan Stopword

In [44]:
# Bahasa Indonesia
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Bahasa Inggris
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Inisialisasi stopword remover
stopword_remover_id = StopWordRemoverFactory().create_stop_word_remover()
stop_words_en = set(stopwords.words('english'))

# Fungsi deteksi bahasa sederhana berdasarkan karakteristik kata
def detect_language(text):
    # Jika banyak kata umum bahasa Indonesia, anggap Bahasa Indonesia
    indo_keywords = ['dan', 'di', 'yang', 'untuk', 'dengan']
    count = sum(1 for word in text.lower().split() if word in indo_keywords)
    return 'id' if count > 1 else 'en'

# Fungsi gabungan penghapusan stopword
def remove_stopwords(text):
    lang = detect_language(text)
    if lang == 'id':
        return stopword_remover_id.remove(text)
    elif lang == 'en':
        words = word_tokenize(text)
        filtered = [word for word in words if word.lower() not in stop_words_en and word.isalnum()]
        return ' '.join(filtered)
    else:
        raise ValueError("Deteksi bahasa gagal. Pastikan teks menggunakan Bahasa Indonesia atau Inggris.")

# Terapkan langsung pada kolom Deskripsi (tanpa buat kolom baru)
df['Deskripsi_Stop'] = df['Deskripsi_Norm'].apply(remove_stopwords)

# Tampilkan hasil
print("Hasil Penghapusan Stopword (Otomatis ID/EN):")
display(df[['Deskripsi_Norm', 'Deskripsi_Stop']])


🔹 Hasil Penghapusan Stopword (Otomatis ID/EN):


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,Deskripsi_Norm,Deskripsi_Stop
0,we are looking for a data scientist who is equ...,looking data scientist equally comfortable wri...
1,career growth opportunity in a rapidly growing...,career growth opportunity rapidly growing company
2,minimum 5 years experience as data scientist s...,minimum 5 years experience data scientist stro...
3,you will use data to understand user needs and...,use data understand user needs defacto voice c...
4,budaya kerja kekeluargaan,budaya kerja kekeluargaan
...,...,...
103,pengalaman pekerjaan bidang it selama minimal ...,pengalaman pekerjaan bidang selama minimal 7 t...
104,cybertrend as one of key player data science a...,cybertrend one key player data science ai comp...
105,at cybertrend we believe that every company ca...,cybertrend believe every company unlock full e...
106,finns worlds best beach club is more than a be...,finns worlds best beach club beach club destin...


## Stemming

In [45]:
# Bahasa Indonesia
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Bahasa Inggris
from nltk.stem import SnowballStemmer

# Inisialisasi stemmer
stemmer_id = StemmerFactory().create_stemmer()
stemmer_en = SnowballStemmer("english")

# Fungsi deteksi bahasa (sama seperti sebelumnya)
def detect_language(text):
    indo_keywords = ['dan', 'di', 'yang', 'untuk', 'dengan']
    count = sum(1 for word in text.lower().split() if word in indo_keywords)
    return 'id' if count > 1 else 'en'

# Fungsi stemming gabungan
def stem_text(text):
    lang = detect_language(text)
    if lang == 'id':
        return stemmer_id.stem(text)
    elif lang == 'en':
        words = word_tokenize(text)
        stemmed = [stemmer_en.stem(word) for word in words if word.isalnum()]
        return ' '.join(stemmed)
    else:
        raise ValueError("Deteksi bahasa gagal. Pastikan teks menggunakan Bahasa Indonesia atau Inggris.")

# Terapkan langsung ke kolom Deskripsi
df['Deskripsi_Stem'] = df['Deskripsi_Stop'].apply(stem_text)

# Tampilkan hasil
print("Hasil Stemming (Otomatis ID/EN):")
display(df[['Deskripsi_Stop', 'Deskripsi_Stem']])


Hasil Stemming (Otomatis ID/EN):


,Deskripsi_Stop,Deskripsi_Stem
0,looking data scientist equally comfortable wri...,look data scientist equal comfort write sql qu...
1,career growth opportunity rapidly growing company,career growth opportun rapid grow compani
2,minimum 5 years experience data scientist stro...,minimum 5 year experi data scientist strong pr...
3,use data understand user needs defacto voice c...,use data understand user need defacto voic cus...
4,budaya kerja kekeluargaan,budaya kerja kekeluargaan
...,...,...
103,pengalaman pekerjaan bidang selama minimal 7 t...,pengalaman pekerjaan bidang selama minim 7 tuj...
104,cybertrend one key player data science ai comp...,cybertrend one key player data scienc ai compa...
105,cybertrend believe every company unlock full e...,cybertrend believ everi compani unlock full ex...
106,finns worlds best beach club beach club destin...,finn world best beach club beach club destin v...


In [51]:
# Tampilkan Data hasil processing
df_data = display(df[['Posisi', 'Perusahaan', 'Lokasi', 'Gaji', 'Waktu', 'Deskripsi_Stem']])
df_data

,Posisi,Perusahaan,Lokasi,Gaji,Waktu,Deskripsi_Stem
0,Data Scientist,PT Ninety Nine Dotco,Jakarta Raya,Gaji tidak disebutkan,5 hari yang lalu,look data scientist equal comfort write sql qu...
1,Junior Data Scientist,PT Fata Organa Solusi,Tangerang,Rp 8.500.000 – Rp 10.500.000 per month,1 hari yang lalu,career growth opportun rapid grow compani
2,Senior Data Scientist,PT Kamoro Maxima Integra,Jakarta Raya,Gaji tidak disebutkan,9 hari yang lalu,minimum 5 year experi data scientist strong pr...
3,Data Scientist (Analytics),PT Solusi Transportasi Indonesia,Jakarta Raya,Gaji tidak disebutkan,16 hari yang lalu,use data understand user need defacto voic cus...
4,Data Analyst,Pengiklan Anonim,Tangerang,Gaji tidak disebutkan,5 hari yang lalu,budaya kerja kekeluargaan
...,...,...,...,...,...,...
103,Data & Application Operation Center,PT Jaya Teknik Indonesia,Jakarta Raya,Gaji tidak disebutkan,25 hari yang lalu,pengalaman pekerjaan bidang selama minim 7 tuj...
104,Product Head,PT Cybertrend Intrabuana,Pasar Minggu,Gaji tidak disebutkan,22 hari yang lalu,cybertrend one key player data scienc ai compa...
105,Product Innovation Excellence Head,PT Cybertrend Intrabuana,Pasar Minggu,Gaji tidak disebutkan,22 hari yang lalu,cybertrend believ everi compani unlock full ex...
106,Senior Technology Architect,Finns Beach Club,Badung,Gaji tidak disebutkan,6 hari yang lalu,finn world best beach club beach club destin v...


In [ ]:
# Menyimpan file full
df_data.to_csv('Full_Data_Job_Listings_Clean.csv', index=False)